<a href="https://colab.research.google.com/github/sambhuyadav/Credit-Card-Fraud-Detection-Model/blob/main/MLFinalProject(CreditCardFraudDetection).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [141]:
credit_card_data = pd.read_csv("/content/creditcard.csv")

In [142]:
credit_card_data.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [143]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [144]:
# Observing the distribution of the legit transactions & Fradulent transaction i.e Class in our data.
credit_card_data['Class'].value_counts()


,count
Class,
0,284315
1,492


0-> Legitimate Transaction
1-> Fradulent Transaction

As from the distribution of the data, we see there is huge unbalance of the data. Legit transaction is way too high than the fradulent data.

In [145]:
Legit = credit_card_data[credit_card_data.Class == 0]
Fraud = credit_card_data[credit_card_data.Class == 1]

In [146]:
Legit["Amount"].describe()

,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [147]:
Fraud['Amount'].describe()

,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [148]:
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Dealing with the unbalanced data.

To deal with the unbalanced data we will undersample the data with the similar distribution of normal transactions and Fradulent transactions.

Since we have the Fradulent Transaction data less which is only 492. So we will randomly choose only 492 rows from the legitimate transaction to build our model.

In [149]:
Legit_Sample = Legit.sample(n=492)

In [150]:
print(Legit_Sample.shape)
print(Fraud.shape)

(492, 31)
(492, 31)


In [151]:
Balanced_Credit_Card_Data = pd.concat([Legit_Sample, Fraud], axis=0)

In [152]:
Balanced_Credit_Card_Data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
184289,126182.0,2.140954,0.274712,-2.893636,0.328503,1.232903,-0.955630,0.728578,-0.423948,0.164263,...,-0.018104,0.095943,-0.120896,0.002821,0.487793,0.722762,-0.107130,-0.054606,18.10,0
138168,82516.0,-0.960988,1.177700,-0.199856,-1.401786,2.454515,3.351393,0.306550,0.745088,0.198931,...,-0.426865,-0.944120,-0.057611,0.972087,0.094926,0.071932,0.417264,0.091079,9.85,0
206836,136394.0,2.272184,-1.262014,-2.932695,-1.716743,0.148663,-0.968035,0.124757,-0.589540,-2.061317,...,0.126200,0.673123,-0.275077,0.084596,0.711613,0.304631,-0.091396,-0.072365,108.90,0
224319,143736.0,2.161951,-0.993776,-0.739137,-1.161209,-0.614410,0.224223,-1.113654,0.139882,-0.147596,...,-0.037184,-0.169209,0.328072,0.063086,-0.425824,-0.499756,0.002459,-0.048783,14.98,0
51500,44981.0,-3.816057,-2.172202,1.695160,0.997538,2.360042,-0.330723,1.732917,-2.319840,2.471674,...,-1.503421,0.461674,1.261781,-0.153402,-0.002074,0.262385,-0.971824,-1.863793,11.10,0


In [153]:
Balanced_Credit_Card_Data['Class'].value_counts()

,count
Class,
0,492
1,492


In [154]:
Balanced_Credit_Card_Data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,98064.174797,-0.084221,0.018145,-0.019956,0.011797,0.027185,-0.093411,-0.027301,0.089346,-0.057296,...,0.050177,0.010334,-0.002337,-0.003620,0.025938,-0.010609,-0.001719,-0.001321,0.006312,84.399126
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Though, we have undersampled the data the mean of the each column has not significantly changed. This ensures that the nature of the data has not changed.

In [155]:
X = Balanced_Credit_Card_Data.drop(columns='Class', axis=1)
Y = Balanced_Credit_Card_Data['Class']

In [156]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
184289,126182.0,2.140954,0.274712,-2.893636,0.328503,1.232903,-0.955630,0.728578,-0.423948,0.164263,...,-0.127081,-0.018104,0.095943,-0.120896,0.002821,0.487793,0.722762,-0.107130,-0.054606,18.10
138168,82516.0,-0.960988,1.177700,-0.199856,-1.401786,2.454515,3.351393,0.306550,0.745088,0.198931,...,0.668100,-0.426865,-0.944120,-0.057611,0.972087,0.094926,0.071932,0.417264,0.091079,9.85
206836,136394.0,2.272184,-1.262014,-2.932695,-1.716743,0.148663,-0.968035,0.124757,-0.589540,-2.061317,...,-0.247711,0.126200,0.673123,-0.275077,0.084596,0.711613,0.304631,-0.091396,-0.072365,108.90
224319,143736.0,2.161951,-0.993776,-0.739137,-1.161209,-0.614410,0.224223,-1.113654,0.139882,-0.147596,...,0.036716,-0.037184,-0.169209,0.328072,0.063086,-0.425824,-0.499756,0.002459,-0.048783,14.98
51500,44981.0,-3.816057,-2.172202,1.695160,0.997538,2.360042,-0.330723,1.732917,-2.319840,2.471674,...,-2.244590,-1.503421,0.461674,1.261781,-0.153402,-0.002074,0.262385,-0.971824,-1.863793,11.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00


In [157]:
Y

,Class
184289,0
138168,0
206836,0
224319,0
51500,0
...,...
279863,1
280143,1
280149,1
281144,1


Splitting the data into Training & Testing data.

In [158]:
X_train, x_test, Y_train, y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

Logistic Regression

In [159]:
model = LogisticRegression()

In [160]:
model.fit(X_train, Y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Model Evaluation

Accuracy Score

In [163]:
X_train_prediction = model.predict(X_train)
train_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [170]:
train_data_accuracy

0.9428208386277002

Checking Accuracy on Test Data

In [168]:
X_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [171]:
test_data_accuracy

0.9441624365482234